# Demo of the game

Import necessary files

In [1]:
import time

# Code fromPython files
from tile import Tile as TP
from generation import find_all_valid_moves
from generation import possible_moves

# Code from C++ files
import rummikub_solver as rs
from rummikub_solver import Tile as TC
from rummikub_solver import TileColor as Color

Create hand and table examples

In [2]:
hand = [TP(7, 'Red'), TP(7, 'Blue')]
table = [
    [TP(4, 'Red'), TP(5, 'Red'), TP(6, 'Red')],
    [TP(4, 'Blue'), TP(5, 'Blue'), TP(6, 'Blue')]
]

Find all possible moves

In [3]:
possible = find_all_valid_moves(hand, table)

print(f"\nZnaleziono {len(possible)} możliwych ruchów:")


Znaleziono 1 możliwych ruchów:


In [4]:
for i, move in enumerate(possible):
    # Sorting for readability of the result
    sorted_move = sorted([sorted(meld) for meld in move])
    print(f"--- Ruch #{i+1} ---")
    for meld in sorted_move:
        print(f"  {meld}")

--- Ruch #1 ---
  [B4, B5, B6, B7]
  [R4, R5, R6, R7]


Now more complex example

In [5]:
hand = [TP(6, 'Black')]
table = [
    [TP(4, 'Red'), TP(4, 'Blue'), TP(4, 'Yellow'), TP(4,'Black')],
    [TP(5, 'Black'), TP(5, 'Yellow'), TP(5, 'Red'), TP(5, 'Blue')]
]

In [6]:
possible = find_all_valid_moves(hand, table)

print(f"\nZnaleziono {len(possible)} możliwych ruchów:")


Znaleziono 1 możliwych ruchów:


In [7]:
for i, move in enumerate(possible):
    sorted_move = sorted([sorted(meld) for meld in move])
    print(f"--- Ruch #{i+1} ---")
    for meld in sorted_move:
        print(f"  {meld}")

--- Ruch #1 ---
  [B4, B5, B6]
  [B4, R4, Y4]
  [B5, R5, Y5]


With repeating blocks

In [8]:
hand = [TP(5, 'Red'), TP(5, 'Red')]
table = [
    [TP(3, 'Red'), TP(4, 'Red'), TP(6, 'Red'), TP(7, 'Red')],
]

In [9]:
possible = find_all_valid_moves(hand, table)

print(f"\nZnaleziono {len(possible)} możliwych ruchów:")


Znaleziono 1 możliwych ruchów:


In [10]:
for i, move in enumerate(possible):
    sorted_move = sorted([sorted(meld) for meld in move])
    print(f"--- Ruch #{i+1} ---")
    for meld in sorted_move:
        print(f"  {meld}")

--- Ruch #1 ---
  [R3, R4, R5]
  [R5, R6, R7]


With jokers

In [11]:
hand = [TP(1, 'Joker'), TP(2, 'Joker'), TP(1, 'Yellow')]
table = []

In [12]:
possible = find_all_valid_moves(hand, table)

print(f"\nZnaleziono {len(possible)} możliwych ruchów:")


Znaleziono 1 możliwych ruchów:


In [13]:
for i, move in enumerate(possible):
    sorted_move = sorted([sorted(meld) for meld in move])
    print(f"--- Ruch #{i+1} ---")
    for meld in sorted_move:
        print(f"  {meld}")

--- Ruch #1 ---
  [J1, Y1, J2]


### Compare with C++ implementation
Create identical hand and table

In [14]:
hand_CPP = [TC(7, Color.Red), TC(7, Color.Blue), TC(7, Color.Blue)]
table_CPP = [
    [TC(5, Color.Red), TC(6, Color.Red), TC(8, Color.Red), TC(9, Color.Red)],
    [TC(5, Color.Blue), TC(6, Color.Blue), TC(8, Color.Blue), TC(9, Color.Blue)]
]

In [15]:
hand_Python = [TP(7, "Red"), TP(7, "Blue"), TP(7, "Blue")]
table_Python = [
    [TP(5, "Red"), TP(6, "Red"), TP(8, "Red"), TP(9, "Red")],
    [TP(5, "Blue"), TP(6, "Blue"), TP(8, "Blue"), TP(9, "Blue")]
]

count time in both implementations

In [16]:
start_python = time.time()
possible_python = find_all_valid_moves(hand_Python, table_Python)
end_python = time.time()
python_time = end_python - start_python

In [17]:
start_cpp = time.time()
possible_cpp = rs.find_all_valid_moves(hand_CPP, table_CPP)
end_cpp = time.time()
cpp_time = end_cpp - start_cpp

compare results and times

In [18]:
print(f"Python found {len(possible_python)} and C++ found {len(possible_cpp)}")
print(f"Python time: {python_time:.6f} seconds")
print(f"C++ time: {cpp_time:.6f} seconds")

Python found 1 and C++ found 1
Python time: 0.000553 seconds
C++ time: 0.000173 seconds


With larger numbers

In [19]:
hand_python = [TP(12, 'Black'), TP(13, 'Black'), TP(11, 'Black'), TP(2, 'Black'), TP(3, 'Black'), TP(4, 'Black'), TP(5, 'Black'), TP(6, 'Black'), TP(7, 'Black'),
        TP(1, 'Red'), TP(2, 'Red'), TP(3, 'Red'), TP(4, 'Red'), TP(5, 'Red'), TP(6, 'Red'), TP(7, 'Red')]
table_python = [
    [TP(12, 'Black'), TP(13, 'Black'), TP(11, 'Black')],
    [TP(2, 'Black'), TP(3, 'Black'), TP(4, 'Black'), TP(5, 'Black'), TP(6, 'Black'), TP(7, 'Black')],
    [TP(1, 'Red'), TP(2, 'Red'), TP(3, 'Red'), TP(4, 'Red'), TP(5, 'Red'), TP(6, 'Red'), TP(7, 'Red')],
    [TP(1, 'Blue'), TP(2, 'Blue'), TP(3, 'Blue'), TP(4, 'Blue'), TP(5, 'Blue'), TP(6, 'Blue'), TP(7, 'Blue')],
    [TP(1, 'Yellow'), TP(2, 'Yellow'), TP(3, 'Yellow'), TP(4, 'Yellow'), TP(5, 'Yellow'), TP(6, 'Yellow'), TP(7, 'Yellow')]
]

In [20]:
hand_CPP = [TC(12, Color.Black), TC(13, Color.Black), TC(11, Color.Black), TC(2, Color.Black), TC(3, Color.Black), TC(4, Color.Black), TC(5, Color.Black), TC(6, Color.Black), TC(7, Color.Black),
        TC(1, Color.Red), TC(2, Color.Red), TC(3, Color.Red), TC(4, Color.Red), TC(5, Color.Red), TC(6, Color.Red), TC(7, Color.Red)]
table_CPP = [
    [TC(12, Color.Black), TC(13, Color.Black), TC(11, Color.Black)],
    [TC(2, Color.Black), TC(3, Color.Black), TC(4, Color.Black), TC(5, Color.Black), TC(6, Color.Black), TC(7, Color.Black)],
    [TC(1, Color.Red), TC(2, Color.Red), TC(3, Color.Red), TC(4, Color.Red), TC(5, Color.Red), TC(6, Color.Red), TC(7, Color.Red)],
    [TC(1, Color.Blue), TC(2, Color.Blue), TC(3, Color.Blue), TC(4, Color.Blue), TC(5, Color.Blue), TC(6, Color.Blue), TC(7, Color.Blue)],
    [TC(1, Color.Yellow), TC(2, Color.Yellow), TC(3, Color.Yellow), TC(4, Color.Yellow), TC(5, Color.Yellow), TC(6, Color.Yellow), TC(7, Color.Yellow)]
]

In [21]:
start_python = time.time()
possible_python = find_all_valid_moves(hand_python, table_python)
end_python = time.time()
python_time = end_python - start_python

In [22]:
start_cpp = time.time()
possible_cpp = rs.find_all_valid_moves(hand_CPP, table_CPP)
end_cpp = time.time()
cpp_time = end_cpp - start_cpp

In [23]:
print(f"Python found {len(possible_python)} and C++ found {len(possible_cpp)}")
print(f"Python time: {python_time:.6f} seconds")
print(f"C++ time: {cpp_time:.6f} seconds")

Python found 3909 and C++ found 3909
Python time: 29.875791 seconds
C++ time: 1.948364 seconds
